In [1]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]])#입력(2차원), 원-핫 표현
W = np.random.randn(7,3)#가중치
h = np.matmul(c,W)#중간 노드
print(h)

[[-0.18545942 -0.76276988  0.74948955]]


In [3]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

In [2]:
#Matmul계층으로 수행
import sys
sys.path.append('..')
from common.layers import MatMul

c = np.array([[1,0,0,0,0,0,0]])#입력
W = np.random.randn(7,3)#가중치
layer = MatMul(W)#Matmul 계층
h = layer.forward(c)#순전파 수행
print(h)

[[-0.46922656  0.07682951  1.20890558]]


In [11]:
#CBOW(continuous bag of words) 모델의 추론 처리

#샘플 맥락 데이터
c0 = np.array([[1,0,0,0,0,0,0]])#you
c1 = np.array([[0,0,1,0,0,0,0]])#goodbye

#가중치 초기화
W_in = np.random.randn(7,3)#입력-은닉 가중치
W_out = np.random.randn(3,7)#은닉-출력 가중치

#계층 생성
in_layer0 = MatMul(W_in)#첫번째 Matmul 계층
in_layer1 = MatMul(W_in)#두번째 Matmul 계층
out_layer = MatMul(W_out)#은닉층 Matmul

#순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = .5*(h0 +h1)#평균내기
s = out_layer.forward(h)#출력층

print(s)#점수
print(np.argmax(s))#할 때마다 점수가 바뀌네..?

[[ 0.42206677  1.06581229  1.40990315  0.76112996 -0.52625162  1.23304314
  -1.54347268]]
2


 학습 데이터 준비


In [12]:
#말뭉치 텍스트를 단어 ID로 변환
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)#단어 ID의 배열

print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [13]:
corpus.shape

(8,)

In [17]:
def create_contexts_target(corpus, window_size = 1):#맥락과 타깃을 만드는 함수 구현
    '''
    인수
    corpus: 단어 ID의 배열
    window_size: 맥락의 윈도우 크기
    '''
    target = corpus[window_size:-window_size]#1차원 배열
    contexts = []
    
    for idx in range(window_size, len(corpus) - window_size):
        cs = []
        for t in range(-window_size, window_size+1):
            if t == 0:
                continue
            cs.append(corpus[idx+t])
        contexts.append(cs)
        
    return np.array(contexts), np.array(target)

In [19]:
#함수 사용
contexts, target = create_contexts_target(corpus, window_size=1)
print("contexts:\n",contexts)

print("\n\ntarget:\n",target)

contexts:
 [[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]


target:
 [1 2 3 4 1 5]


In [20]:
#원-핫 표현으로 변환
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환
    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

In [21]:
text

'You say goodbye and I say hello.'

In [23]:
vocab_size = len(word_to_id)
vocab_size

7

In [24]:
target = convert_one_hot(target, vocab_size)
target

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0]])

In [25]:
contexts

array([[0, 2],
       [1, 3],
       [2, 4],
       [3, 1],
       [4, 5],
       [1, 6]])

In [26]:
contexts = convert_one_hot(contexts, vocab_size)

In [27]:
contexts

array([[[1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0]],

       [[0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1]]])

In [29]:
#CBOW 모델 구현
import sys
sys.path.append('..')
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):#어휘수, 은닉층의 뉴런수
        V, H = vocab_size, hidden_size
        
        #가중치 초기화(무작위 생성)
        W_in = 0.01*np.random.randn(V, H).astype('f')#32비트 부동소수점 수로 초기화
        W_out = 0.01*np.random.randn(H, V).astype('f')
        
        #계층 생성
        self.in_layer0 = MatMul(W_in)#입력층
        self.in_layer1 = MatMul(W_in)#입력층
        self.out_layer = MatMul(W_out)#출력층
        self.loss_layer = SoftmaxWithLoss()#활성화 + 손실 층
        
        #모든 가중치와 기울기를 리스트에 모은다
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [],[]
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
        #인스턴스 변수에 단어의 분산 표현을 저장한다
        self.word_vecs = W_in
        
    def forward(self, contexts, target):#순전파
        h0 = self.in_layer0.forward(contexts[:,0])#입력
        h1 = self.in_layer1.forward(contexts[:,1])#입력
        h = (h0+h1)*0.5#평균
        score = self.out_layer.forward(h)#점수
        loss = self.loss_layer.forward(score, target)#손실
        return loss#손실 반환